# EDA - NYC Airbnb
This notebook fixes a few problems in the data:
- Remove outliers on `price` column
- Convert `last_review` to datetime

In [1]:
import wandb
import pandas as pd
import pandas_profiling
import hydra

## Initialise a run on Weights & Biases

Name the project as `nyc_airbnb` and group these runs into `eda`.\
The `save_code` is necessary to upload and version this notebook.

In [2]:
run = wandb.init(project="nyc_airbnb", group="eda", save_code=True)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mfurquim. Use `wandb login --relogin` to force relogin


## Retrieve data from Weights & Biases

Retrieve `sample.csv` and load into a pandas DataFrame

In [3]:
local_path = wandb.use_artifact("sample.csv:latest").file()
df = pd.read_csv(local_path)
df.head(1)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,9138664,Private Lg Room 15 min to Manhattan,47594947,Iris,Queens,Sunnyside,40.74271,-73.92493,Private room,74,2,6,2019-05-26,0.13,1,5


## First glance at data

Run a simple describe of data and a better analysis with pandas_profiling

In [4]:
df.shape

(20000, 16)

In [5]:
df.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,2.000000e+04,2.000000e+04,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,15877.000000,20000.000000,20000.000000
mean,1.892380e+07,6.746034e+07,40.728455,-73.952125,153.269050,6.992100,23.274100,1.377446,6.955450,112.901200
std,1.101223e+07,7.857936e+07,0.054755,0.046559,243.325609,21.645449,44.927793,1.683006,32.433831,131.762226
min,2.539000e+03,2.571000e+03,40.508730,-74.239140,0.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,9.393540e+06,7.853718e+06,40.689420,-73.983030,69.000000,1.000000,1.000000,0.190000,1.000000,0.000000
50%,1.952117e+07,3.111431e+07,40.722730,-73.955640,105.000000,2.000000,5.000000,0.720000,1.000000,44.000000
75%,2.912936e+07,1.068426e+08,40.762990,-73.936380,175.000000,5.000000,23.000000,2.010000,2.000000,229.000000
max,3.648561e+07,2.742733e+08,40.913060,-73.717950,10000.000000,1250.000000,607.000000,27.950000,327.000000,365.000000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              20000 non-null  int64  
 1   name                            19993 non-null  object 
 2   host_id                         20000 non-null  int64  
 3   host_name                       19992 non-null  object 
 4   neighbourhood_group             20000 non-null  object 
 5   neighbourhood                   20000 non-null  object 
 6   latitude                        20000 non-null  float64
 7   longitude                       20000 non-null  float64
 8   room_type                       20000 non-null  object 
 9   price                           20000 non-null  int64  
 10  minimum_nights                  20000 non-null  int64  
 11  number_of_reviews               20000 non-null  int64  
 12  last_review                     

In [7]:
pandas_profiling.ProfileReport(df).to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

## Fixing problems
- Drop duplicates if any
- Remove `price` outliers
- Convert `last_review` to datetime

### Drop duplicates if any

In [8]:
print(f"Shape before dropping duplicates: {df.shape}")

df.drop_duplicates(inplace=True)
      
print(f"Shape after dropping duplicates:  {df.shape}")

Shape before dropping duplicates: (20000, 16)
Shape after dropping duplicates:  (20000, 16)


### Convert `last_review` str to datetime

In [9]:
print(f"last_review type before: {type(df.loc[0, 'last_review'])}")

df['last_review'] = pd.to_datetime(df['last_review'])

print(f"last_review type after:  {type(df.loc[0, 'last_review'])}")

last_review type before: <class 'str'>
last_review type after:  <class 'pandas._libs.tslibs.timestamps.Timestamp'>


### We do not imput missing values
But if we were going to do it, maybe we could do the following

In [10]:
# df['last_review'] = pd.to_datetime(df['last_review']).fillna(pd.to_datetime(df['last_review']).min())
# df['reviews_per_month'].fillna(0, inplace=True)

### Drop `price` outliers

In [11]:
min_price = 10
max_price = 350
idx = df['price'].between(min_price, max_price)
df = df[idx].copy()

## Understand how the data was changed

In [12]:
df.shape

(19001, 16)

In [13]:
df.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,1.900100e+04,1.900100e+04,19001.000000,19001.000000,19001.000000,19001.000000,19001.000000,15243.000000,19001.000000,19001.000000
mean,1.883041e+07,6.639459e+07,40.728063,-73.950827,122.340456,6.906900,23.797747,1.380928,6.583811,109.725383
std,1.096986e+07,7.782663e+07,0.055389,0.046825,71.530346,21.456544,45.493455,1.689988,31.154750,130.599899
min,2.539000e+03,2.571000e+03,40.508730,-74.239140,10.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,9.355498e+06,7.728754e+06,40.688820,-73.982050,66.000000,1.000000,1.000000,0.190000,1.000000,0.000000
50%,1.938754e+07,3.048785e+07,40.721710,-73.954630,100.000000,2.000000,6.000000,0.720000,1.000000,39.000000
75%,2.891952e+07,1.048354e+08,40.763210,-73.934490,160.000000,5.000000,24.000000,2.010000,2.000000,219.000000
max,3.648561e+07,2.742733e+08,40.913060,-73.717950,350.000000,1250.000000,607.000000,27.950000,327.000000,365.000000


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19001 entries, 0 to 19999
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              19001 non-null  int64         
 1   name                            18994 non-null  object        
 2   host_id                         19001 non-null  int64         
 3   host_name                       18993 non-null  object        
 4   neighbourhood_group             19001 non-null  object        
 5   neighbourhood                   19001 non-null  object        
 6   latitude                        19001 non-null  float64       
 7   longitude                       19001 non-null  float64       
 8   room_type                       19001 non-null  object        
 9   price                           19001 non-null  int64         
 10  minimum_nights                  19001 non-null  int64         
 11  nu

In [15]:
profile = pandas_profiling.ProfileReport(df)
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

## Finish Weights & Biases run

In [16]:
run.finish()